# Embedding sample notebook

In [ ]:
from powerbi_client import Report

In [ ]:
import requests

# Embed config for sample report
res = requests.get('https://playgroundbe-bck-1.azurewebsites.net/Reports/SampleReport')

access_token = res.json()['EmbedToken']['Token']
embed_url = res.json()['EmbedUrl']

### Create an instance of Power BI Report

In [ ]:
report = Report(access_token, embed_url, token_type=1)

### Power BI event handlers

In [ ]:
def loaded_callback(event_details):
    print('loaded')

def rendered_callback(event_details):
    print('rendered')

def error_callback(event_details):
    print('error', event_details)

report.on('loaded', loaded_callback)
report.on('rendered', rendered_callback)
report.on('error', error_callback)

#### Load the report in the output cell

In [ ]:
# Render report
report

#### Change dimensions of embedded report

In [ ]:
# Change dimensions of embedded report
report.set_dimensions(300, 450)

### Export data from a visual of the embedded report

In [ ]:
page_name = 'ReportSection600dd9293d71ade01765'
visual_name = 'VisualContainer1'

exported_data_task = report.extract_data(page_name, visual_name, rows=5)

# This doesn't work (blocks the kernel indefinitely)
# exported_data = await report._extract_data(page_name, visual_name, rows=5)

In [ ]:
# Get the result after task is finished
exported_data = exported_data_task.result()

print(exported_data)

In [ ]:
page_name = 'ReportSection600dd9293d71ade01765'
visual_name = 'VisualContainer1'

exported_data_task2 = report.extract_data(page_name, visual_name, rows=4)

In [ ]:
exported_data_task2.result()